# 【問題1】ラグランジュの未定乗数法による最急降下
SVMの学習は、ラグランジュの未定乗数法を用います。サンプル数分のラグランジュ乗数 
λを用意して、以下の式により更新していきます。この計算を行うメソッドをScratchSVMClassifierクラスに実装してください。

$$
λ
n
e
w
i
=
λ
i
+
α
(
1
−
n
∑
j
=
1
λ
j
y
i
y
j
k
(
x
i
,
x
j
)
)
$$ 
ここで 
k
(
x
i
,
x
j
)
 はカーネル関数です。線形カーネルの場合は次のようになります。他のカーネル関数にも対応できるように、この部分は独立したメソッドとしておきましょう。

$$
k
(
x
i
,
x
j
)
=
x
T
i
x
j
$$
条件として、更新毎に 
λ
i
>=
0
を満たす必要があります。満たさない場合は 
λ
i
=
0
とします。


i
,
j
 : サンプルのインデックス


λ
n
e
w
i
 : 更新後のi番目のサンプルのラグランジュ乗数


λ
i
 : 更新前のi番目のサンプルのラグランジュ乗数


α
 : 学習率


λ
j
 : j番目のサンプルのラグランジュ乗数


y
i
 : i番目のサンプルのラベル


y
j
 : j番目のサンプルのラベル


x
i
 : i番目のサンプルの特徴量ベクトル


x
j
 : j番目のサンプルの特徴量ベクトル


あるサンプルに対しての全てのサンプルとの関係を計算していくことになります。



In [372]:
class ScratchSVMClassifier():
    """
    SVM分類器のスクラッチ実装
    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    kernel : str
      カーネルの種類。線形カーネル（linear）か多項式カーネル（polly）
    threshold : float
      サポートベクターを選ぶための閾値
    verbose : bool
      学習過程を出力する場合はTrue
    Attributes
    ----------
    self.n_support_vectors : int
      サポートベクターの数（線を引く時の最小値の数）
    self.index_support_vectors : 次の形のndarray, shape (n_support_vectors,)
      サポートベクターのインデックス
    self.X_sv :  次の形のndarray, shape(n_support_vectors, n_features)
      サポートベクターの特徴量
    self.lam_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターの未定乗数
    self.y_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターのラベル
    """
    def __init__(self, num_iter, lr, kernel='linear', threshold=1e-5, verbose=False):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.lr = lr
        self.kernel = kernel
        self.threshold = threshold
        self.verbose = verbose
        self.support_vector = []
        
    def fit(self, X, y, X_val=None, y_val=None):
        """
        SVM分類器を学習する。検証データが入力された場合はそれに対する精度もイテレーションごとに計算する。
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """
        self.lam_sv = np.random.rand(X.shape[0], 1)
        
        for i in range(self.iter):
            self._upgrade_svm(X, y)
            #lam_sv_max = np.max()
            
            if self.verbose:
                #verboseをTrueにした際は学習過程を出力
                print('{}/{} lamda{}'.format(i+1, self.iter, self.lam_sv.max()))
        
         # サポートベクターを決定しインデックスとして保存する
        self.support_vector = np.where(self.lam_sv >= self.threshold)
        # サポートベクターの2次元のインデックスを保存する
        # 問題3の計算をここで保存する（yの値が必要なのでpredictで実装すると引数にyが必要）
        support_vector = self.support_vector[0]
        print("suport vector", support_vector)
        print("suport vector", support_vector.shape)
        self.kernel_sv =  np.dot(X, X[support_vector].T) # カーネル(100,2) (2,100) 転置必要？？
        self.calucation = self.lam_sv[support_vector].T*y[support_vector] #λ*y
        print("kernel_sv", self.kernel_sv)
        print("kernel_sv", self.kernel_sv.shape)
        print("suport vector", self.calucation)
        print("suport vector", self.calucation.shape)
        
        
    def predict(self, X):
        """
        SVM分類器を使いラベルを推定する。
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル
        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            SVM分類器による推定結果
        """

        f_x = np.dot(self.calucation.T, self.kernel_sv)
        f_x = np.sign(f_x)
        return f_x # -1、１のどちらかを出力する
    
    def _upgrade_svm(self, X, y):
        # y_dot = (100,100) 
        # self.lam_sv = (100,1) 
        #np.dot(self.lam_sv, y_dot) = (100,1)
        # _kernel(X) = (100,100) 
        np.random.seed(0)
        y = y.reshape(-1,1)
        y_dot = np.dot(y, y.T) # (100,1) (1,100)
        self.lam_sv = self.lam_sv + self.lr*(1 - np.dot(self.lam_sv.T, (y_dot*kernel(X)))) #計算の順番注意
        self.lam_sv = np.where(self.lam_sv<=0, 0, self.lam_sv) # ０以下を０に変換する
        
    def _kernel(self, X):
        kernel = np.dot(X, X.T)  # (100,1), X(1,100) (100,100)
        return kernel
    
    def _sign(self, X):
        w = np.dot(np.dot(self.lam_sv, y).T, X) #λ(100,100) y(100,1) X(100,2) = (1,2) 
        y_sign = np.sign(w, X.T) # (1,2) (2, 100) wを転置するならwの計算前から
        return y_sign # (1,100)
    

In [373]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

model_sc = ScratchSVMClassifier(1000,0.0001, verbose=None)

model_sc.fit(X_train,y_train)

suport vector [ 0  0  0 ... 74 74 74]
suport vector (2889,)
kernel_sv [[ 2.41  2.41  2.41 ...  7.35  7.35  7.35]
 [ 2.07  2.07  2.07 ...  6.3   6.3   6.3 ]
 [ 6.82  6.82  6.82 ... 20.85 20.85 20.85]
 ...
 [ 1.88  1.88  1.88 ...  5.7   5.7   5.7 ]
 [ 6.48  6.48  6.48 ... 19.8  19.8  19.8 ]
 [ 7.35  7.35  7.35 ... 22.5  22.5  22.5 ]]
kernel_sv (75, 2889)
suport vector [[0.         0.         0.         ... 0.83926358 0.83926358 0.83926358]
 [0.         0.         0.         ... 0.83926358 0.83926358 0.83926358]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.83926358 0.83926358 0.83926358]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
suport vector (75, 2889)


In [376]:
model_sc.predict(X_test)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [308]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

np.printoptions(threshold=None)

iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
X = X.loc[:, ['petal length (cm)', 'petal width (cm)']][0:100].to_numpy()
y = iris.target[0:100]

a = np.random.rand(X.shape[0], 1)
b = np.arange(0,12).reshape(3,4)
c = np.arange(0,15).reshape(3,5)
where = np.where(c%2==0)
print(where)
print(c)
c[where]
y.reshape(-1,1)[where[0]]

(array([0, 0, 0, 1, 1, 2, 2, 2]), array([0, 2, 4, 1, 3, 0, 2, 4]))
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [269]:
def gradient(X, y, lr=0.0001, threshold=1e-5):
    # y_dot = (100,100) 
    # self.lam_sv = (100,1) 
    #np.dot(self.lam_sv, y_dot) = (100,1)
    # _kernel(X) = (100,100) 
    np.random.seed(0)
    lam_sv = np.random.rand(X.shape[0], 1)
    y = y.reshape(-1,1)
    
    y_dot = np.dot(y, y.T) # (100,1) (1,100)  (100,100)
    lam_sv = lam_sv + lr*(1 - np.dot(lam_sv.T, (y_dot*kernel(X))))
    lam_sv = np.where(lam_sv <= 0, 0, lam_sv)
    support_vector = np.where(lam_sv >= threshold)
    return lam_sv, support_vector


def kernel(X):
    kernel = np.dot(X, X.T)  # (100,2), X(2,100) (100,100)
    return kernel

a, b = gradient(X,y)
a

array([[0.5489135 , 0.5489135 , 0.5489135 , ..., 0.50731877, 0.51935739,
        0.50908534],
       [0.71528937, 0.71528937, 0.71528937, ..., 0.67369464, 0.68573325,
        0.6754612 ],
       [0.60286338, 0.60286338, 0.60286338, ..., 0.56126865, 0.57330726,
        0.56303521],
       ...,
       [0.02020755, 0.02020755, 0.02020755, ..., 0.        , 0.        ,
        0.        ],
       [0.82904003, 0.82904003, 0.82904003, ..., 0.7874453 , 0.79948392,
        0.78921187],
       [0.00479548, 0.00479548, 0.00479548, ..., 0.        , 0.        ,
        0.        ]])

# 【問題2】サポートベクターの決定
計算したラグランジュ乗数 $\lambda$ が設定した閾値より大きいサンプルをサポートベクターとして扱います。推定時にサポートベクターが必要になります。サポートベクターを決定し、インスタンス変数として保持しておくコードを書いてください。


閾値はハイパーパラメータですが、1e-5程度からはじめると良いでしょう。サポートベクターの数を出力させられるようにしておくと学習がうまく行えているかを確認できます。



# 【問題3】推定
推定時には、推定したいデータの特徴量とサポートベクターの特徴量をカーネル関数によって計算します。求めた 
f
(
x
)
 の符号が分類結果です。

# 【問題4】学習と推定
機械学習スクラッチ入門のSprintで用意したシンプルデータセット1の2値分類に対してスクラッチ実装の学習と推定を行なってください。


scikit-learnによる実装と比べ、正しく動いているかを確認してください。


AccuracyやPrecision、Recallなどの指標値はscikit-learnを使用してください。

# 【問題5】決定領域の可視化
決定領域を可視化してください。


以下の例のようにサポートベクターは異なる色で示してください。

# 【問題6】（アドバンス課題）多項式カーネル関数の作成
最初に作成した実装では線形カーネルを使用していました。多項式カーネルにも切り替えられるようにしましょう。